# Scripts for grading MTH 337 reports

In [48]:
import numpy as np
import pandas as pd
import json
import os
import os.path
import re
import shutil
import openpyxl
from email.message import EmailMessage
import smtplib
from time import sleep
import getpass

# path to the Excel worksheet with grades; {} will be replaced by the project number
grade_file_template = "./grades_{}.xlsx"
# name of the sheet in the Excel worksheet with student grades
sheet = "GRADES"
# column number in the worksheet with person numbers
pnum_col = 0
# column number in the worksheet with UNIT names
ubit_col = 1
# column number in the worksheet with names of students
name_col = 2
# column number in the worksheet with first names of students - used to format email messages
fname_col = 3
# the number of the first row with student data (row numbering starts with 1!)
min_row = 3
# the number of the last row with student data (row numbering starts with 1!)
max_row = 100

# path to directory with submited files;  {} will be replaced by the project number
submission_path_template = "./project_{}_submissions"
# path to directory with graded files;  {} will be replaced by the project number
# this directory will be automatically created if it does not exist
graded_path_template = "./project_{}_graded/"


# for sending emails:
# UB mail server
smtp_server = "smtp.buffalo.edu"

# CHANGE! 
login_name = "badzioch@buffalo.edu"
instructor_email = "badzioch@buffalo.edu"
instructor_name = "Bernard Badzioch"

# CHANGE! 
#path to the file with highlighter css file
highligh_css_file = '/Users/bb/Library/Jupyter/nbextensions/highlighter/highlighter.css'






def get_fnames(num, printout=True):
    '''
    gets names of submission files
    num = project number
    ''' 
    
    global submission_path_template
    global grade_file_template
    global sheet
    global ubit_col
    global min_row
    global max_row
    
    path = submission_path_template.format(num)
    grade_file = grade_file_template.format(num)
    files_dict = {}
    
    # get ubit names from the grades spreadsheet
    grades = openpyxl.load_workbook(grade_file, data_only="True", read_only = True)[sheet] 
    all_ubits = [r[ubit_col].value for r in grades.iter_rows(min_row=min_row, max_row=max_row)]
    
    #parse text files downloaded from UBLearns
    tfnames = os.listdir(path)
    for tfname in [tfname for tfname in tfnames if tfname.endswith(".txt")]:
        with open(path +'/' + tfname, 'r') as tfile:
            text = tfile.read()
            if not text.startswith("Name:"): 
                continue
            name, ubit = re.findall(r"Name: (.*) \((.*)\)\n", text)[0]
            text_dict = files_dict[ubit] = {"name":name}
            submission_text = re.findall(r"Submission Field:(.*)Comments:", text, re.S)[0].strip()
            if "There is no student submission text data for this assignment." in submission_text:
                submission_text = None
            text_dict["submission"] = submission_text
            comments_text = re.findall(r"Comments:(.*)Files", text, re.S)[0].strip()
            if "There are no student comments for this assignment." in comments_text:
                comments_text = None
            text_dict["comments"] = comments_text
            submitted_files = re.findall(r"Original filename: (.*?)\s*?Filename: (.*?)\n\n", text, re.S)
            text_dict["files"] = submitted_files

    ubits = sorted(list(files_dict.keys()))
    if printout:
        print("Number of submissions: {}\n".format(len(files_dict)))
        i = 1
        for ubit in ubits:
            text_dict = files_dict[ubit]
            print("{}.".format(i))
            i += 1
            print("{:12} {}".format("UBIT:", ubit))
            print("{:12} {}".format("NAME:", text_dict["name"]))
            print("{:12} {}".format("COMMENTS:", text_dict["comments"] if text_dict["comments"] else ''))
            print("{:12} {}".format("SUBMISSION:", text_dict["submission"] if text_dict["submission"] else ''))
            print("FILES:")
            for file in text_dict["files"]:
                print(file[1])
            print('')
    
    print("Submission missing:")
    print("-------------------")
    for n in all_ubits:
        if n not in ubits:
            print(n)
    print('')
                
    return files_dict




def clean_fnames(files_dict, num):
    '''
    clean names of submission files
    file_dict = dictionary of names of submission files
    num = project number
    ''' 
    
    global submission_path_template
    global graded_path_template
    
    path = submission_path_template.format(num)
    new_path = graded_path_template.format(num)
    
    if not os.path.isdir(new_path):
        os.makedirs(new_path)
    
    no_notebook = []
    for ubit in files_dict:
        done = False
        for file in files_dict[ubit]["files"]:
            fname = file[1]
            if fname.endswith(".ipynb"):
                shutil.copy(path +'/' + fname, new_path + '/' + ubit + '_project_' + str(num) + '.ipynb')
                done = True
        if not done:
            no_notebook.append(ubit)
    print(".ipynb file missing:")
    print("--------------------")
    for n in sorted(no_notebook):
        print(n)

        
        
        
def format_table(row, weights):
    '''
    format HTML grade table
    row = an array with student scores and instructor comments
    weights = an array listing weight of each grade component
    '''
     
    # format code in instructor's comments
    # *{code}* 
    row = [s.replace('*{', '<code style="color:red;white-space:pre;font-style:normal;">') for s in row]
    row = [s.replace('}*', '</code>') for s in row]
    
    lname, fname = row[2].split(",")
    name = f"{fname} {lname}"
           

    
    table =[
        r'<p style="text-align: left; color: red; font-size: 180%; font-weight: bold;">',
        r'</p>',
        r'<table  style="width:100%; table-layout: fixed; border-collapse: collapse; border: 2px solid red; font-size: 110%; color:red; align=left;"><tr style="border: 2px solid red; background-color:Gainsboro"><th style="text-align: left; border: 2px solid red;">Introduction Section: ',
        r'</th><th style="text-align: left; border: 2px solid red;">Conclusions Section: ',
        r'</th><th style="text-align: left; border: 2px solid red;">Content: ',
        r'</th><th style="text-align: left; border: 2px solid red;">Total: ',
        r'</th></tr><tr><td style="border: 2px solid red; text-align: left; font-size: 80%; font-style:italic; vertical-align: text-top;">',
        r'</td><td style="border: 2px solid red; text-align: left; font-size: 80%; font-style:italic; vertical-align: text-top;">',
        r'</td><td style="border: 2px solid red; text-align: left; font-size: 80%; font-style:italic; vertical-align: text-top;">',
        r'</td><td rowspan="3"; style="border: 2px solid red; text-align: left; font-size: 80%; font-style:italic; vertical-align: text-top;">',
        r'</td></tr><tr style="border: 2px solid red; background-color:Gainsboro"><th style="text-align: left; border: 2px solid red;">Code: ',
        r'</th><th style="text-align: left; border: 2px solid red;">Presentation: ',
        r'</th><th style="text-align: left; border: 2px solid red;">Bonus: ',
        r'</th></tr><tr><td style="border: 2px solid red; text-align: left; font-size: 80%; font-style:italic; vertical-align: text-top;">',
        r'</td><td style="border: 2px solid red; text-align: left; font-size: 80%; font-style:italic; vertical-align: text-top;">',
        r'</td><td style="border: 2px solid red; text-align: left; font-size: 80%; font-style:italic; vertical-align: text-top;">',
        r'</td></tr></table>'
       ]
    
    
    score_table = (  table[0]
                   + name + " " + row[0]                    # student name & person number
                   + table[1]                                
                   + table[2]                                # table header
                   + row[4] + ' (' + weights[2] + '%)'       # Intro score
                   + table[3]
                   + row[6] + ' (' + weights[4] + '%)'       # Conclusions score
                   + table[4]
                   + row[8] + ' (' + weights[6] + '%)'       # Content score
                   + table[5]
                   + row[16]                                 # Total score
                   + table[6]
                   + row[5]                                  # Intro comment
                   + table[7]
                   + row[7]                                  # Conclusions comment
                   + table[8]
                   + row[9]                                  # Content comment
                   + table[9]
                   + row[17]                                 # Total comment
                   + table[10]
                   + row[10] + ' (' + weights[8] + '%)'       # Code score
                   + table[11]
                   + row[12] + ' (' + weights[10] + '%)'     # Presentation score
                   + table[12]
                   + row[14]                                 # Bonus score
                   + table[13]
                   + row[11]                                  # Code comment
                   + table[14]
                   + row[13]                                 # Presentation comment
                   + table[15]
                   + row[15]                                 # Bonus comment
                   + table[16]
                   )
    
    return score_table

def insert_markdown(nb_dict, cell_num, new_text):
    '''
    Takes as the argument a dictionary representing Jupyter notebook
    and inserts in it a new markdown cell number cell_num containing new_text
    '''
    notebook = nb_dict.copy()
    notebook['cells'].insert(cell_num, {'cell_type': 'markdown', 'metadata': {}, 'source': [new_text]})  
    return notebook

import nbconvert
import nbformat

def notebook_to_html(nb, template='full'):
    """
    Converts a notebook to HTML. template can be either 'basic' (default) or 'full'. 
    Returns a string with the HTML code.
    """
    nb = json.dumps(nb)
    nnode = nbformat.reads(nb, as_version=nbformat.NO_CONVERT)
    exporter = nbconvert.HTMLExporter(config={'Exporter': {'template_file': template}})
    html_str = exporter.from_notebook_node(nnode)[0]
    return html_str





def grading_xlsx(num):
    '''
    Arguments:
    num: number of the report (needed to get names of report files)
    '''
    

    global grade_file_template
    global graded_path_template
    global sheet
    global ubit_col
    global pnum_col
    global name_col
    global min_row
    global max_row
    global highligh_css_file
    
    
    
    grade_file = grade_file_template.format(num)
    graded_path = graded_path_template.format(num)
    files_dict = {}
    
    # highlighting css
    with open(highligh_css_file, 'r') as hcss:
        highlighter = '<style>' + (hcss.read()).replace('\n', ' ') + '</style>'
    
    
    grades = openpyxl.load_workbook(grade_file, data_only="True", read_only = True)[sheet]      

    #the row with weights of report components 
    weights = [c.value for c in grades[2]]    
    weights = ['' if s == None else str(s) for s in weights]

    
    for r in grades.iter_rows(min_row = min_row, max_row = max_row):
        
        if r[0].value == None:
            continue

        row = list([c.value for c in r])
        name = r[name_col].value
        ubit = r[ubit_col].value
        # report grade
        final_grade = str(row[16])
        
        row = ['' if s == None else str(s) for s in row]
        table = format_table(row, weights)
        
        # path to notebook file
        nb_infile = graded_path + "{}_project_{}.ipynb".format(ubit, num)
        # path to HTML file
        html_outfile = graded_path + "{}_project_{}_graded.html".format(ubit, num)
        
        
        if not os.path.isfile(nb_infile):
            print('NOT FOUND: {0:>10}_project_{1}  {2:30} ***********'.format(ubit, num, name) )
            continue 
        
        

        if final_grade != '#N/A':
            graded = 'GRADED'
            
            #open notebook
            with open(nb_infile, 'r') as jfile:
                nb_dict = json.load(jfile)
        
        
            # insert grade table
            notebook = insert_markdown(nb_dict, 0, table)
        
            # add highlighting css in the last cell
            notebook = insert_markdown(notebook, -1, highlighter)


            # convert to html
            html_code = notebook_to_html(notebook)


            # save html file
            with open(html_outfile, 'w') as html_file:
                html_file.write(html_code)
        else:
            graded = ''
        
        s = 'Finished:  {0:>10}_project_{1}  {2:30} {3}: {4:3}'.format(ubit, num, name, graded, final_grade)
        print(s)
        
        
def make_message(num, fname, ubit):
    '''
    This function returns an e-message which is to be send to a student. 
    The message consists of a text and an attachment with a graded homework 
    assignment (which here is an html file). 
    
    Arguments:
    
    num    = number of the homework assignment
    fname  = the first name of the recipient
    ubit   = the ubit name of the recipient
    '''
    
    global graded_path_template
    global instructor_email
    global instructor_name

    
    # directory where graded assignment files are stored 
    graded_path = graded_path_template.format(num)
    
    # e-mail address of the recipient
    #s_email = ubit + "@buffalo.edu"
    
    # template of the email text
    msg_txt = ("Dear {fname},\n\n"
               "I am attaching an html file containing your graded MTH 337 Project Report {num}.\n"
               "You can open it in any web browser.\n"
               "\nBest regards,\n\n"
               "{instructor_name}\n\n")
    
    # construct the path to the graded assignment file and check if the file exists
    # if it does not exist return -1
    html_file = "{}_project_{}_graded.html".format(ubit, num)
    html_path = graded_path + html_file
    if not os.path.isfile(html_path):
        return -1
    
    # construct the email message object
    msg = EmailMessage()
    msg['Subject'] = "MTH 337 Project Report {}".format(num)
    msg['From'] = instructor_email
    msg['To'] = s_email
    
    # set the text of the message 
    msg['Content-Type'] = "text/plain; charset=utf-8; format=flowed"
    text = msg_txt.format(fname = fname, num = num, instructor_name = instructor_name)
    msg.set_content(text)
    
    # attach the html file with the graded assignment to the email
    with open(html_path, 'rb') as f:
        msg.add_attachment(f.read(), maintype = "text", subtype = "html", filename = html_file)
            
    # this fixes some issues with MIME headers in the email message
    body = msg.get_body()
    body.replace_header('Content-Type', 'text/plain; charset=utf-8; format=flowed') 
    for p in msg.walk():
        if p['Content-Type'] == 'text/html':
            p.replace_header('Content-Type', 'text/html; name={}'.format(html_file))
            break
    
    return msg


def timer(seconds):
    """ 
    Countdown timer. 
    Used by the function send_emails temporarily pause sending emails if server rejects new messages. 
    """

    for i in range(seconds,0,-1):
        print("\rWill retry in {:3} seconds.".format(i), end='')
        sleep(1)
        sl = len("Will retry in {:3} seconds.".format(i))
        print("\r" + " "*sl + "\r", end='')


        
def send_emails(num):
    '''
    This function sends email to student with their graded assignments
    
    Arguments:
    
    num = the number of the assignment
    '''

    global grade_file_template
    global sheet
    global ubit_col
    global name_col
    global fname_col
    global min_row
    global max_row
    
    global smtp_server
    global login_name  
    global instructor_email
    global instructor_name
    
    # the duration of time (in seconds) of the pause in sending 
    # messages if the server rejects additional emails 
    reconnect_period = 300
    
    # open the excel spreadsheet with student grades
    grade_file = grade_file_template.format(num) 
    grades = openpyxl.load_workbook(grade_file, data_only="True", read_only = True)[sheet] 
    
    # get the password to the UB email account - needed for sending messages though the UB server 
    instructor_password = getpass.getpass(login_name + " password:")
    
    # iterate over rows with student data in the spreadsheet
    for r in grades.iter_rows(min_row=min_row, max_row=max_row):
        
        # skip over rows with no UBIT name
        if r[ubit_col].value == None:
            continue
        else:
            # get student name, first name, UBIT name
            name = r[name_col].value
            fname = r[fname_col].value
            ubit = r[ubit_col].value
            
            # student email address
            #s_email = ubit + "@buffalo.edu"
            
            # comstruct the email message
            msg = make_message(num, fname, ubit)
            
            # do not send an email if a student did not submit an assignment
            if msg == -1:
                print('{:30} *********** FILE NOT FOUND, NOT SENT'.format(name))
                continue 
            
            #sent the email
            send_success = False
            while not send_success:       
                try:
                    server = smtplib.SMTP_SSL(smtp_server, 465)
                    server.login(login_name, instructor_password)
                    server.send_message(msg, from_addr = instructor_email, to_addrs = s_email)
                except smtplib.SMTPException as ex:
                    print("{:30} *********** NOT SENT: {}".format(name, ex))
                    #pause before reattempting to send the message 
                    timer(reconnect_period)
                else:
                    print("{:30} *********** SENT".format(name))
                    send_success = True
                    server.quit()
                    sleep(0.1)
    print("***FINISHED***")

## Step 0

* Download the zipped file with students' submissions from UBLearns, unzip it, rename the unzipped folder 
`project_N_submissions` where `N` is the number of the project, and  place it in the directory where this notebook is located.



* Make a copy of the `grades_template.xlsx` file, rename it `grades_N.xlsx` where `N` is the number of the project, and place it in the directory where this notebook is located. 

## Step 1

This function creates a dictionary with  data about  project files submitted by students. The first argument is the number of the project.

In [49]:
d = get_fnames(1, printout=False)

Submission missing:
-------------------
kelliweg
slmcgrat
yzhang99



## Step 2

This function prepares notebook files for grading. Files are copied the `project_n_graded` where `n` is the project number. The first function argument is the directory created in Step 1, the second argument is the project number.

In [52]:
clean_fnames(d, 1)

.ipynb file missing:
--------------------


In [51]:
d["jblewis6"]

{'name': 'Jordana Lewis',
 'submission': None,
 'comments': None,
 'files': [('p01_50222929.ipynb',
   'Project 1_jblewis6_attempt_2019-09-15-21-09-05_p01_50222929.ipynb')]}

## Step 3

This function  inserts the table with grades into reports and converts them into html files. These files are saved in the  `project_n_graded` directory. The function argument is the project number. 

In [34]:
grading_xlsx(1)

Finished:    allababi_project_1  Babiker,Alla Abdelwahab        GRADED: A  
Finished:    bgkessle_project_1  Kessler,Bradley G              GRADED: A  
Finished:     cefinn2_project_1  Finn,Colleen Elizabeth         GRADED: B+ 
Finished:        csw8_project_1  Wilson,Connor                  GRADED: B+ 
Finished:    easchlan_project_1  Schlant,Elizabeth Anne         GRADED: A  
Finished:    evanmaes_project_1  Maestri,Evan Vincent           : #N/A
Finished:    evargas4_project_1  Vargas Jr,Ernesto              GRADED: A  
Finished:    guozheng_project_1  Kang,Guozheng                  GRADED: A  
Finished:    isaacvil_project_1  Villacis,Isaac A               GRADED: A  
Finished:    jacobbol_project_1  Bolton,Jacob A                 GRADED: B+ 
NOT FOUND:   jblewis6_project_1  Lewis,Jordana Brooke           ***********
Finished:     jcdrozd_project_1  Drozd,Jonathan Charles         GRADED: B+ 
Finished:    jjhoefen_project_1  Hoefenkrieg,Joe                GRADED: A+ 
NOT FOUND:   kell

# Step 4

This function sends graded html files to students. It will ask for UB password in order to access the instructor email account. The function argument is the report number. 

In [35]:
send_emails(1)

badzioch@buffalo.edu password:········
Babiker,Alla Abdelwahab        *********** SENT
Kessler,Bradley G              *********** SENT
Finn,Colleen Elizabeth         *********** SENT
Wilson,Connor                  *********** SENT
Schlant,Elizabeth Anne         *********** SENT
Maestri,Evan Vincent           *********** FILE NOT FOUND, NOT SENT
Vargas Jr,Ernesto              *********** SENT
Kang,Guozheng                  *********** SENT
Villacis,Isaac A               *********** SENT
Bolton,Jacob A                 *********** SENT
Lewis,Jordana Brooke           *********** FILE NOT FOUND, NOT SENT
Drozd,Jonathan Charles         *********** SENT
Hoefenkrieg,Joe                *********** SENT
Wegner,Kelli A                 *********** FILE NOT FOUND, NOT SENT
Khan,Nashmia                   *********** FILE NOT FOUND, NOT SENT
Alfahal,Nasreen                *********** FILE NOT FOUND, NOT SENT
Forth,Orandy Oren              *********** FILE NOT FOUND, NOT SENT
Rai,Rajat                